In [ ]:
from db.db import backup_database

backup_destination = r"N:\PPDO\BS\Records Department\Archive_Data"
backup_database(backup_dir=backup_destination, compress=True)

In [ ]:
from sqlalchemy.orm import Session
from sqlalchemy import or_
import pandas as pd
import numpy as np
from db import get_db_engine
from db.models import FileLocation, File, FileContent
from text_extraction.pdf_extraction import PDFTextExtractor
from embedding.minilm import MiniLMEmbedder
from utils import extract_server_dirs, build_file_path
server_mount = r"N:\PPDO\Records"
target_filepath = r"N:\PPDO\Records\23xx   Crown College (College 3)\2318\2318\H - Submittals\Lumber_ProductData.pdf"
target_location = r"N:\PPDO\Records\37xx   Oakes College\3701\3701"


pdf_extractor = PDFTextExtractor()
embedder = MiniLMEmbedder()

text = pdf_extractor(target_filepath)
embeddings = embedder.encode(text)
target_loc_dirs = extract_server_dirs(full_path=target_location, base_mount=server_mount)
engine = get_db_engine()

target_sim_distance = lambda file_embeds: 1 - np.dot(file_embeds, embeddings)
build_local_path = lambda row: build_file_path(base_mount=server_mount, server_dir=row['file_server_directories'], filename=row['filename'])

with Session(engine) as db_session:
    files_located_in_dir = or_(
        FileLocation.file_server_directories == target_loc_dirs,
        FileLocation.file_server_directories.startswith(target_loc_dirs + '/')
    )

    query = db_session.query(FileLocation, FileContent)

In [ ]:
# This cell is to backfill the text_length column in FileContent for existing rows where source_text is present.

from db.models import FileContent
from db.db import get_db_engine
from sqlalchemy.orm import sessionmaker

# Create a session
engine = get_db_engine()
Session = sessionmaker(bind=engine)
session = Session()

# Query all FileContent rows where text_length is null and source_text is not null
to_update = session.query(FileContent).filter(
    (FileContent.text_length.is_(None)) & (FileContent.source_text.isnot(None))
).all()

print(f"Found {len(to_update)} rows to backfill.")

for fc in to_update:
    fc.text_length = len(fc.source_text)

session.commit()
print("Backfill complete.")

session.close()

In [1]:
from sqlalchemy import create_engine, text
import os
import dotenv
import pandas as pd
from utils import extract_server_dirs, build_file_path
dotenv.load_dotenv()

server_mount = r"N:\PPDO\Records"
build_local_path = lambda row: build_file_path(base_mount=server_mount, server_dir=row['file_server_directories'], filename=None)    

engine = create_engine(
    f"postgresql+psycopg://{os.getenv('APP_DB_USERNAME')}:{os.getenv('APP_DB_PASSWORD')}"
    f"@{os.getenv('APP_DB_HOST')}:{os.getenv('APP_DB_PORT')}/{os.getenv('APP_DB_NAME')}"
)

g22_query = """
SELECT *
FROM file_locations
WHERE file_server_directories LIKE '%/G22%'
"""

g22_df = pd.read_sql(text(g22_query), engine)

# remove rows with duplicat file_server_directories values, keeping the first occurrence
g22_df = g22_df.drop_duplicates(subset=['file_server_directories'])
g22_df['local_path'] = g22_df.apply(build_local_path, axis=1)

# now we need to create a column which counts the number of file_locations entries which have the values in file_server_directories
def location_count(row):
    dir_path = row['file_server_directories']
    # query database for count of entries with this directory path
    count_query = text("""
    SELECT COUNT(*)
    FROM file_locations
    WHERE file_server_directories = :dir_path
    """)
    with engine.connect() as conn:
        result = conn.execute(count_query, {"dir_path": dir_path})
        count = result.scalar()
    return count

g22_df['files_in_location'] = g22_df.apply(location_count, axis=1)

# remove unnecessary columns
g22_df = g22_df[['file_server_directories', 'local_path', 'files_in_location']]
g22_df.head()


,file_server_directories,local_path,files_in_location
0,26xx Merrill College (College 4)/2638/2638/G...,N:\PPDO\Records\26xx Merrill College (Colleg...,24
2,27xx Applied Sciences Baskin Engineering/270...,N:\PPDO\Records\27xx Applied Sciences Baskin...,5
18,26xx Merrill College (College 4)/2638/2638/G...,N:\PPDO\Records\26xx Merrill College (Colleg...,16
26,26xx Merrill College (College 4)/2638/2638/G...,N:\PPDO\Records\26xx Merrill College (Colleg...,15
39,26xx Merrill College (College 4)/2638/2638/G...,N:\PPDO\Records\26xx Merrill College (Colleg...,9


In [2]:
dirs = g22_df['file_server_directories']
direct_counts = g22_df['files_in_location']

def recursive_sum(dir_path: str) -> int:
    prefix = dir_path + '/'
    mask = (dirs == dir_path) | dirs.str.startswith(prefix)
    return int(direct_counts[mask].sum())

g22_df['files_recursive'] = dirs.map(recursive_sum)

# (Optional) rename for clarity
g22_df = g22_df.rename(columns={'files_in_location': 'files_direct'})

# Export
g22_df.to_excel(r"C:\Users\adankert\Downloads\g22_file_locations.xlsx", index=False)
g22_df.head()

,file_server_directories,local_path,files_direct,files_recursive
0,26xx Merrill College (College 4)/2638/2638/G...,N:\PPDO\Records\26xx Merrill College (Colleg...,24,24
2,27xx Applied Sciences Baskin Engineering/270...,N:\PPDO\Records\27xx Applied Sciences Baskin...,5,5
18,26xx Merrill College (College 4)/2638/2638/G...,N:\PPDO\Records\26xx Merrill College (Colleg...,16,31
26,26xx Merrill College (College 4)/2638/2638/G...,N:\PPDO\Records\26xx Merrill College (Colleg...,15,15
39,26xx Merrill College (College 4)/2638/2638/G...,N:\PPDO\Records\26xx Merrill College (Colleg...,9,9
